In [106]:
import os
from tqdm import tqdm
from datasets import load_dataset
import sys
sys.path.append('../evaluation')
from evaluate import RetrievalSystem, main as evaluate_main
import yaml
import hyde
import json
from vector_store import EmbeddingClient, Document, DocumentLoader
import filters
import hyde_reranking
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import semantic_search

/Users/christineye/anaconda3/envs/jsalt-retrieval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/christineye/anaconda3/envs/jsalt-retrieval/lib/python3.10/site-packages


In [99]:
test_hyde = semantic_search.EmbeddingRetrievalSystem(weight_citation = True, weight_date= True, weight_keywords= True)

Loading embeddings...
Loading documents...
Loading index mapping...
Processing document dates...
Loading metadata...
Data loaded successfully.
Loading filters...
Loading existing index...
Index loaded successfully.


In [107]:
test_hyde = hyde_reranking.HydeCohereRetrievalSystem(max_doclen = 100, weight_citation = False, weight_keywords = False, weight_date = True)

Loading embeddings...
Loading documents...
Loading index mapping...
Processing document dates...
Loading metadata...
Data loaded successfully.
Loading filters...


In [119]:
query = "What was the observed stellar mass of the Milky Way as of 2010?"
arxiv_id = "2301.00001"
top_k = 10

In [115]:
import numpy as np

In [120]:
results = test_hyde.retrieve(query, arxiv_id, top_k, reweight = True)
print(results)

{'has_temporal_aspect': True, 'expected_year_filter': 'year == 2010', 'expected_recency_weight': 6}
['1002.1775', '1003.0280', '1002.2009', '1005.4593', '1009.2843', '1009.4857', '1006.1281', '1010.5273', '1011.3815', '1011.3037']


In [86]:
test_hyde.keyword_filter = KeywordFilter(index_path = "../data/vector_store/keyword_index.json", metadata = test_hyde.metadata, remove_capitals = True)

Loading existing index...
Index loaded successfully.


In [95]:
test_hyde.keyword_filter.build_index()

Building index...


271540it [00:12, 22534.72it/s]


In [84]:
test_hyde.metadata['1912.04296']['keyword_search']

['total mass',
 'rotation curve data selection',
 'Gaia DR2 rotation curve data',
 'various possible systematics',
 'good statistical agreement',
 'Bayesian model averaging',
 'uncertainties',
 'uncertainty',
 'the dark matter profile assumptions',
 'systematic error',
 'different assumptions',
 'M_\\odot}$',
 'M_{200}^{\\rm DM}=8.3^{+1.2}_{-0.9}{\\rm(stat)}\\times10^{11}\\,{\\rm',
 'surface stellar density and microlensing measurements',
 'the dark matter profile',
 'the total stellar mass',
 'the dark matter distribution',
 'priors',
 'Bayesian',
 'DM}=8.3^{+1.2}_{-0.9}{\\rm(stat)}\\times10^{11}\\,{\\rm']

In [11]:
astro_meta = load_dataset("JSALT2024-Astro-LLMs/astro_paper_corpus", split = "train")

Generating train split: 100%|██████████| 271544/271544 [00:05<00:00, 54228.25 examples/s]


In [36]:
keys = list(paper.keys())
keys.remove('abstract')
keys.remove('introduction')
keys.remove('conclusions')

In [42]:
metadata = {}
for paper in astro_meta:
    id_str = paper['arxiv_id']
    metadata[id_str] = {key: paper[key] for key in keys}

In [13]:
with open('../data/vector_store/metadata.json', 'r') as f:
    metadata = json.load(f)

In [28]:
def make_keyword_index(metadata):
    keyword_index = {}
    
    for i, index in tqdm(enumerate(metadata)):
        paper = metadata[index]
        for keyword in paper['keyword_search']:
            term = ' '.join(word for word in keyword.lower().split() if word.lower() not in stopwords)
            if term not in keyword_index:
                keyword_index[term] = []
            
            keyword_index[term].append(paper['arxiv_id'])
    
    return keyword_index

In [29]:
keyword_index = make_keyword_index(metadata)

271540it [00:11, 24324.32it/s]


In [69]:
with open('../data/vector_store/keyword_index.json', 'w') as f:
    json.dump(keyword_index, f)

In [47]:
test = hyde_reranking.HydeCohereRetrievalSystem(config_path = "../config.yaml")

Loading embeddings...
Loading documents...
Loading index mapping...
Processing document dates...
Loading metadata...
Data loaded successfully.


In [57]:
test.weight_citation = False
test.retrieve(query, arxiv_id, top_k)

['0801.1023',
 '1711.01453',
 '2101.05821',
 '1001.3411',
 '1503.06065',
 '1703.08585',
 '1611.01545',
 '1510.06665',
 '2003.04925',
 '1908.00116']

In [56]:
query = "What are the primary computational methods used in modern cosmological simulations, and what are some notable examples of each approach?"
test.weight_citation = True
test.retrieve(query, arxiv_id, top_k)

['0801.1023',
 '1711.01453',
 '2101.05821',
 '1001.3411',
 '1703.08585',
 '1510.06665',
 '2003.04925',
 '1908.00116',
 'astro-ph0611863_arXiv.txt',
 'astro-ph0005502_arXiv.txt']

In [21]:
metadata['1210.3157']['keyword_search']

['a cored dark matter halo',
 'a dark matter halo',
 'Fornax',
 'a constant density core',
 'the dark matter mass profile',
 'a core size',
 'the core size',
 'radius',
 'constraints',
 'a dark matter cusp',
 'Spheroidal',
 'order',
 'the Fornax dwarf Spheroidal',
 'first',
 'unrealistically large scale radii',
 'the concentration parameter',
 'Navarro-Frenk-White haloes',
 'its density distribution',
 'Navarro',
 'three distinct stellar subpopulations']

In [135]:
mode = "hyde"

if mode == "semantic":
    retrieval = semantic_search.EmbeddingRetrievalSystem()
elif mode == "hyde":
    retrieval = hyde.HydeRetrievalSystem(config_path="../config.yaml")
elif mode == "hydecohere":
    retrieval = hyde_reranking.HydeCohereRetrievalSystem()
else:
    print("No retrieval system selected.")

Loading embeddings...
Loading documents...
Loading index mapping...
Processing document dates...
Loading metadata...
Data loaded successfully.
Loading filters...


In [199]:
import sciencetree

In [231]:
tree = sciencetree.scienceTreeNode("Seek out nearby worlds and explore their habitability", retrieval)

In [181]:
import sciencetree

In [232]:
sciencetree.print_latex_tree(tree)


\documentclass{article}
\usepackage[landscape]{geometry}
\usepackage{tikz}
\usetikzlibrary{fit, positioning}
\usepackage{forest}
\begin{document}
\centering
\begin{forest}
for tree={
    draw,
    rectangle,
    rounded corners,
    align=center,
    inner sep=2pt,
    anchor=north,
    fit tree
}
[{\node{\parbox{8cm}{Seek out nearby worlds and explore their habitability}}} [{\node{\parbox{4cm}{Conduct a systematic survey of atmospheric water and carbon dioxide abundances for terrestrial exoplanets to test the habitable zone hypothesis}}} [{\node{\parbox{2cm}{Atmospheric water vapor abundance}}} [{\node{\parbox{2cm}{Near-infrared spectroscopy at 0.94 μm to detect the H2O absorption feature}}}] [{\node{\parbox{2cm}{Mid-infrared spectroscopy between 10-36 μm to observe multiple H2O emission lines}}}]] [{\node{\parbox{2cm}{Atmospheric carbon dioxide abundance}}} [{\node{\parbox{2cm}{Infrared spectroscopy in the 12-15 μm wavelength range}}}] [{\node{\parbox{2cm}{Measurements of CO2 mole f

In [225]:
tree.children[0].children